In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [3]:
wikipedia.run("Lords of the ring")

"Page: The Lord of the Rings\nSummary: The Lord of the Rings is an epic high fantasy novel by the English author and scholar J. R. R. Tolkien. Set in Middle-earth, the story began as a sequel to Tolkien's 1937 children's book The Hobbit, but eventually developed into a much larger work. Written in stages between 1937 and 1949, The Lord of the Rings is one of the best-selling books ever written, with over 150 million copies sold.\nThe title refers to the story's main antagonist, the Dark Lord Sauron, who in an earlier age created the One Ring, allowing him to rule the other Rings of Power given to men, dwarves, and elves, in his campaign to conquer all of Middle-earth. From homely beginnings in the Shire, a hobbit land reminiscent of the English countryside, the story ranges across Middle-earth, following the quest to destroy the One Ring, seen mainly through the eyes of the hobbits Frodo, Sam, Merry, and Pippin. Aiding the hobbits are the wizard Gandalf, the men Aragorn and Boromir, th

In [19]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
docs = arxiv.run("1301.3781")
docs

'Published: 2013-09-07\nTitle: Efficient Estimation of Word Representations in Vector Space\nAuthors: Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean\nSummary: We propose two novel model architectures for computing continuous vector\nrepresentations of words from very large data sets. The quality of these\nrepresentations is measured in a word similarity task, and the results are\ncompared to the previously best performing techniques based on different types\nof neural networks. We observe large improvements in accuracy at much lower\ncomputational cost, i.e. it takes less than a day to learn high quality word\nvectors from a 1.6 billion words data set. Furthermore, we show that these\nvectors provide state-of-the-art performance on our test set for measuring\nsyntactic and semantic word similarities.'

In [20]:
author = arxiv.run("Tomas Mikolov")
print(author)

Published: 2016-02-26
Title: A Roadmap towards Machine Intelligence
Authors: Tomas Mikolov, Armand Joulin, Marco Baroni
Summary: The development of intelligent machines is one of the biggest unsolved
challenges in computer science. In this paper, we propose some fundamental
properties these machines should have, focusing in particular on communication
and learning. We discuss a simple environment that could be used to
incrementally teach a machine the basics of natural-language-based
communication, as a prerequisite to more complex interaction with human users.
We also present some conjectures on the sort of algorithms the machine should
support in order to profitably learn from the environment.


In [7]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

wiki.name, print(wiki.run("Boltzmann machine"))

Page: Boltzmann machine
Summary: A Boltzmann machine (also called Sherrington–Kirkpatrick model with external field or stochastic Ising model), named after Ludwig Boltzmann is a spin-glass model with an external field, i.e., a Sherrington–Kirkpatrick


('wikipedia', None)

Retriever as a tool

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACE_API = os.getenv("HF_KEY")

from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
docs = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/").load()
chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(chunks, HuggingFaceEmbeddings())
retriever = vectorstore.as_retriever()
retriever

C:\Users\MercadosEMI\AppData\Local\Temp\ipykernel_33540\2744503018.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = FAISS.from_documents(chunks, HuggingFaceEmbeddings())
C:\Users\MercadosEMI\AppData\Local\Temp\ipykernel_33540\2744503018.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = FAISS.from_documents(chunks, HuggingFaceEmbeddings())
d:\DeepLearningMastery\LLM\Langchain\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEnco

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E09B8B8C0>, search_kwargs={})

In [15]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever=retriever, name="LLM agent search", description="Get any information about LLM agent")

retriever_tool

Tool(name='LLM agent search', description='Get any information about LLM agent', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000012E4A383240>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E09B8B8C0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000012E4A6FD580>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E09B8B8C0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'))

In [23]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\DeepLearningMastery\\LLM\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='LLM agent search', description='Get any information about LLM agent', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000012E4A383240>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E

In [25]:
wiki.name, retriever_tool.name, arxiv.name

('wikipedia', 'LLM agent search', 'arxiv')

In [26]:
# Run these tools with agents and LLM models

from langchain_groq.chat_models import ChatGroq
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="gemma2-9b-it"
)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000012E1FD5CE90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000012E1FD5FF20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [27]:
# Create prompt template

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

prompt, agent, agent_executor

(PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'),
 RunnableAssign(mapper={
   agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']

In [28]:
agent_executor.invoke({
    "input": "What are agents about?"
})



> Entering new AgentExecutor chain...
Thought: I need to learn more about what agents are to answer this question. 
Action: LLM agent search
Action Input: 'agents'}
]
Then after these clarification, the agent moved into the code writing mode with a different system message.
System message:

}
]
Challenges#
After going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:

This benchmark evaluates the agent’s tool use capabilities at three levels:

Level-1 evaluates the ability to call the API. Given an API’s description, the model needs to determine whether to call a given API, call it correctly, and respond properly to API returns.
Level-2 examines the ability to retrieve the API. The model needs to search for possible APIs that may solve the user’s requirement and learn how to use them by reading documentation.
Level-3 assesses the ability to plan API beyond retrieve and call. Given unclear user requests (e.g. schedule group meetin

{'input': 'What are agents about?',
 'output': 'Agents are AI systems designed to act autonomously and make decisions. They can be implemented using various techniques, including reinforcement learning and rule-based systems. \n\nThe provided text describes different aspects of agents, including:\n\n* **Tool use:** Agents can be evaluated on their ability to use tools, such as APIs, to achieve their goals.\n\n* **Risks:** There are concerns about the potential misuse of agents, particularly in areas like illicit drug synthesis.\n\n* **Generative Agents:** A specific type of agent that simulates human behavior in a virtual environment.'}

In [31]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt, prompt.messages

(ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag=

In [32]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(model, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [33]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [35]:
result = agent_executor.invoke({
    "input": "What tasks can agents perform?"
})

result



> Entering new AgentExecutor chain...

Invoking: `LLM agent search` with `{'query': 'What tasks can agents perform?'}`


}
]
Challenges#
After going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.
Long-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.


Tool use

The agent learns to call externa

{'input': 'What tasks can agents perform?',
 'output': 'LLM agents can perform a variety of tasks, including:\n\n**Planning:**\n\n* Breaking down large tasks into smaller, manageable subgoals.\n* Reflecting on past actions and refining them for future steps.\n\n**Memory:**\n\n* Utilizing short-term memory to learn from in-context examples.\n* Retaining and recalling information over extended periods using external vector stores.\n\n**Tool Use:**\n\n* Calling external APIs to access information and functionality not available in their own weights.\n* Searching for and learning to use new APIs based on user requests.\n* Planning and executing complex sequences of API calls to solve multifaceted problems.\n\n\n'}

In [37]:
print(result["output"])

LLM agents can perform a variety of tasks, including:

**Planning:**

* Breaking down large tasks into smaller, manageable subgoals.
* Reflecting on past actions and refining them for future steps.

**Memory:**

* Utilizing short-term memory to learn from in-context examples.
* Retaining and recalling information over extended periods using external vector stores.

**Tool Use:**

* Calling external APIs to access information and functionality not available in their own weights.
* Searching for and learning to use new APIs based on user requests.
* Planning and executing complex sequences of API calls to solve multifaceted problems.





In [38]:
agent_executor.invoke(
    {
        "input": "What is machine learning?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks withhout explicit programming. 


> Finished chain.


{'input': 'What is machine learning?',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks withhout explicit programming. \n'}

In [43]:
result = agent_executor.invoke({
    "input": "What is the way to efficiently estimate the Word Representations in Vector Space"
})

result



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Word Embeddings'}`


Page: Word embedding
Summary: In natural language processing, a word embedding is a representation of a word. The embedding is used in text analysis. Typically, the representation is a real-valued vector that encodes the meaning of the word in such aWord embeddings are typically represented as dense, real-valued vectors. 

Here are some popular and efficient methods for estimating word representations:

* **Word2Vec:** This model learns word embeddings by predicting the context words given a target word (CBOW) or vice versa (Skip-gram). It's known for its efficiency and effectiveness.

* **GloVe (Global Vectors for Word Representation):** GloVe uses global word co-occurrence statistics to learn word embeddings. It considers the global relationships between words in a corpus.

* **FastText:** An extension of Word2Vec that considers subword information, making it more robust to out-of-voc

{'input': 'What is the way to efficiently estimate the Word Representations in Vector Space',
 'output': "Word embeddings are typically represented as dense, real-valued vectors. \n\nHere are some popular and efficient methods for estimating word representations:\n\n* **Word2Vec:** This model learns word embeddings by predicting the context words given a target word (CBOW) or vice versa (Skip-gram). It's known for its efficiency and effectiveness.\n\n* **GloVe (Global Vectors for Word Representation):** GloVe uses global word co-occurrence statistics to learn word embeddings. It considers the global relationships between words in a corpus.\n\n* **FastText:** An extension of Word2Vec that considers subword information, making it more robust to out-of-vocabulary words.\n\nThese methods are generally trained on large text corpora and can be fine-tuned for specific tasks. \n\n\n"}

In [44]:
print(result['output'])

Word embeddings are typically represented as dense, real-valued vectors. 

Here are some popular and efficient methods for estimating word representations:

* **Word2Vec:** This model learns word embeddings by predicting the context words given a target word (CBOW) or vice versa (Skip-gram). It's known for its efficiency and effectiveness.

* **GloVe (Global Vectors for Word Representation):** GloVe uses global word co-occurrence statistics to learn word embeddings. It considers the global relationships between words in a corpus.

* **FastText:** An extension of Word2Vec that considers subword information, making it more robust to out-of-vocabulary words.

These methods are generally trained on large text corpora and can be fine-tuned for specific tasks. 



